In [1]:
import numpy as np
from svgis import *
from IPython.display import display, HTML
from tools.data import *
from tools.geodata import *

In [2]:
#geo = svgis.SVGIS(FILENAME_PATTERN % DATA_FILES["bm-shape"]["filename"])
#geo.compose()

In [3]:
from shapely.geometry import Polygon, MultiPolygon

class GeoDataTest:
    
    def __init__(self, fileid):
        from tools.data import load_json
        self.data = load_json(fileid)

        self.polygons = {}
        for i, f in enumerate(self.data["features"]):
            #points = self._get_points(i)
            poly = self._get_polygon(i)
            if poly:
                self.polygons[f["n"]] = {
                    "polygon": poly,
                    "area": poly.area,
                }

    def _get_points(self, feature_idx):
        # b, d, e, c
        bb = [float(x) for x in self.data["boundingBox"].split()]
        off_x, off_y = bb[:2]
        width, height = bb[2]-bb[0], bb[3]-bb[1]
        pix_w, pix_h = self.data["pixelWidth"], self.data["pixelHeight"]
        points = self.data["features"][feature_idx]["p"][0]
        xa, ya = 0, 0
        ret = []
        for i in range(0, len(points), 2):
            xa += points[i]
            ya += points[i+1]
            ret.append((
                off_x + xa / pix_w * width,
                off_y + ya / pix_h * height
            ))
        return ret

    def _get_polygon(self, feature_idx):
        points = self._get_points(feature_idx)
        if len(points) < 3:
            return None
        return Polygon(points)

    def _get_polygons(self):
        polygons = []
        for i in range(len(self.data["features"])):
            poly = self._get_polygon(i)
            if poly:
                polygons.append(poly)
        return MultiPolygon(polygons)
    
    def polygon_list(self):
        polygons = []
        for n in sorted(self.polygons):
            poly = self.polygons[n].copy()
            poly["id"] = n
            polygons.append(poly)
        return polygons
            
    def to_svg(self, color=None, heatmap=None):
        polygons = self.polygon_list()

        bb = MultiPolygon(p["polygon"] for p in polygons).bounds
        bb = (bb[0], bb[1], bb[2]-bb[0], bb[3]-bb[1])
        svg = """<svg width="400px" height="400px" viewBox="%s" xmlns="http://www.w3.org/2000/svg" version="1.1">""" % (
            " ".join("%s" % x for x in bb),
        )
        
        if heatmap is not None:
            color = dict()
            for poly in polygons:
                color[poly["id"]] = heatmap.get(poly["id"], 0)
            ma = max(color.values())
            if ma:
                color = {k: color[k] / ma for k in color}
            for k in color:
                v = color[k]
                rgb = (v*255, math.pow(v, .3)*255, math.sin(v*3.14)*255)
                color[k] = "rgb(%s, %s, %s)" % tuple((max(0, min(255, x)) for x in rgb))
            
        for poly in polygons:
            svgpoints = " ".join("%s,%s" % (round(t[0]), round(bb[1]+bb[3]-(t[1] - bb[1])))
                                 for t in poly["polygon"].exterior.coords)
            if color is None:
                col = '#ddd'
            elif callable(color):
                col = color(poly["id"])
            elif isinstance(color, dict):
                col = color.get(poly["id"], "#ddd")
            else:
                col = color                
            svg += '<polygon fill="%s" stroke="black" stroke-width="10" points="%s" />' % (
                col, svgpoints
            )

        svg += '</svg>'
        return svg

    def display(self, **kwargs):
        from IPython.display import display, HTML
        display(HTML(self.to_svg(**kwargs)))        

geo = GeoDataTest("bm-shape")
#print(geo.features)
#print([geo.features[f]["area"] for f in geo.features])
geo.display(heatmap={"1": 1, "83": 2})

In [4]:
geo2 = GeoDataTest("stats-shape")
display(HTML('<div style="display: flex; flex-direction: row;">%s %s</div>' % (geo.to_svg(), geo2.to_svg())))

In [5]:
def get_bmwahl_by_stats(bm):
    geo = GeoData("bm-shape")
    geo2 = GeoData("stats-shape")
    
    bm_poly_map = {str(int(k.split()[0])):bm.loc[k] for k in bm.index}
    
    #df = pd.DataFrame(columns=bm.columns, index="Bezirk")
    skey_sum_dic = dict()
    for wkey in geo.polygons:
        wpoly = geo.polygons[wkey]
        row = bm_poly_map[wkey]
        for skey in geo2.polygons:
            spoly = geo2.polygons[skey]
            if spoly["polygon"].intersects(wpoly["polygon"]):
                try:
                    intersection = spoly["polygon"].intersection(wpoly["polygon"])
                    shared_area = intersection.area
                except:
                    shared_area = 0
                if skey not in skey_sum_dic:
                    skey_sum_dic[skey] = [0] * bm.shape[0]
                sums = skey_sum_dic[skey]
                fac = shared_area / wpoly["area"]
                for i, val in enumerate(row):
                    sums[i] += int(round(val * fac))
    
    df = pd.DataFrame({bm.columns[x]: [skey_sum_dic[skey][x] for y, skey in enumerate(sorted(skey_sum_dic))]
                       for x in range(len(bm.columns))},
                      index=sorted(skey_sum_dic), columns=bm.columns)
    return df

bm1 = load_pandas_bmwahl("bm01")
bm1 = rename_bmwahl(bm1)
bm1 = bm1[bm1["n"] != 0]
bms1 = get_bmwahl_by_stats(bm1)
bms1

TopologyException: Input geom 1 is invalid: Self-intersection at or near point 684562.64983339992 5639994.1659012409 at 684562.64983339992 5639994.1659012409
TopologyException: Input geom 1 is invalid: Self-intersection at or near point 684562.64983339992 5639994.1659012409 at 684562.64983339992 5639994.1659012409
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 680635.68085010001 5643501.030052728 at 680635.68085010001 5643501.030052728
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 680635.68085010001 5643501.030052728 at 680635.68085010001 5643501.030052728
TopologyException: Input geom 1 is invalid: Self-intersection at or near point 680960.20962908445 5641403.6809530165 at 680960.20962908445 5641403.6809530165
TopologyException: Input geom 1 is invalid: Self-intersection at or near point 680960.20962908445 5641403.6809530165 at 680960.20962908445 5641403.6809530165
TopologyException: Input geom 1 is invalid: Self-interse

,n,nw,nu,ng,CDU,LINKE,SPD,AFD,GRÜNE,FDP,πRATEN,SANDRO,ARNE
Ammerbach Ort,368,181,0,181,27,11,54,21,6,54,7,0,0
Beutenberg / Winzerlaer Straße,2234,913,3,910,134,95,212,74,60,278,39,12,7
Burgau Ort,348,170,1,169,28,8,43,24,9,41,10,4,1
Closewitz,112,76,0,76,34,0,8,8,3,19,4,0,0
Cospeda,1128,580,2,578,83,46,144,45,26,196,29,7,2
Drackendorf,676,382,1,381,64,26,89,43,12,139,5,2,1
Drackendorf / Lobeda-Ost,2570,998,10,988,152,175,193,103,30,242,33,33,27
Göschwitz,499,255,5,250,40,21,56,29,8,74,13,8,1
Ilmnitz,412,210,1,209,48,7,53,22,4,71,1,3,0
Isserstedt,679,333,3,330,65,27,70,30,20,97,8,10,3


In [6]:
bms1.sum()

n         81501
nw        35153
nu          184
ng        34969
CDU        4870
LINKE      3918
SPD        8543
AFD        2800
GRÜNE      2699
FDP        9354
πRATEN     1603
SANDRO      719
ARNE        457
dtype: int64

In [7]:
bm1 = load_pandas_bmwahl("bm01")
bm1 = rename_bmwahl(bm1)
bm1 = bm1[bm1["n"] != 0]
geo = GeoDataTest("bm-shape")

In [8]:
area_dict = {int(k): geo.polygons[k]["area"] for k in geo.polygons}
areas = pd.Series([area_dict.get(int(k.split()[0]), -1) for k in bm1.index], index=bm1.index)
dens = bm1["n"] / areas 
dens /= max(dens)
#print(dens)
geo.display(heatmap={str(int(k.split()[0])): dens[k] for k in dens.index})


In [9]:
def load_pandas_stat(year=None):
    if year is not None:
        year = str(year)

    data = load_json("stats-index")

    geographies = data["geographies"][0]
    dic = OrderedDict({
        "Bezirk": [f["name"] for f in geographies["features"]],
    })

    for theme in geographies["themes"]:
        url_part = theme["fileName"].split("/")[-1]
        fileid = "stat-%(themeId)s" % theme
        DATA_FILES[fileid] = {
            "filename": "%s.json" % fileid,
            "url": "http://statistiken.jena.de/instantatlas/stadtbezirksstatistik/%s" % url_part
        }
        data = load_json(fileid)
        for indicator in data["indicators"]:
            if year is not None and indicator["date"] != year:
                continue
            dic[("%(name)s(%(date)s)" % indicator).strip()] = [int(x) for x in indicator["values"]]

    df = pd.DataFrame(dic)
    df.index = [x.strip() for x in df["Bezirk"]]
    del df["Bezirk"]
    return df

stats = load_pandas_stat(2016)
stats = stats[stats.index != "nicht zugeordnet"]
#stats.drop()
#print(stats.columns)
geo2 = GeoDataTest("stats-shape")
s = stats["Arbeitslose insgesamt(2016)"].to_dict()
s = {k: int(s[k]) for k in s}
geo2.display(heatmap=s)
stats.sum()

Arbeitslose insgesamt(2016)                                                       3419
Arbeitslose - Männer(2016)                                                        1962
Arbeitslose - Frauen(2016)                                                        1442
Arbeitslose - Deutsche(2016)                                                      2704
Arbeitslose - Ausländer(2016)                                                      707
Arbeitslose - unter 20 Jahre alt(2016)                                              79
Arbeitslose - 20 bis unter 25 Jahre alt(2016)                                      194
Arbeitslose - 55 Jahre und älter(2016)                                             624
SGB II - Arbeitslose insgesamt(2016)                                              2537
SGB II - Arbeitslose - Männer(2016)                                               1496
SGB II - Arbeitslose - Frauen(2016)                                               1035
SGB II - Arbeitslose - Deutsche(2016)      

In [14]:
def calc_stats_percent(stats):
    key = "Einwohner (Hauptwohnung) insgesamt(2016)"
    df = stats.copy()
    # Stimmen für Parteien in Prozent der gültigen Stimmen
    for col in stats.columns:
        if col != key:
            df[col] = df[col] / df[key] * 100
    return df

def get_stats_correlations(stats, bms):
    dic = OrderedDict()
    for wkey in bms.columns: 
        #s = set(stats.index) - set(bms1[wkey].index)
        #print(len(stats.index))
        #print(stats.columns)

        dic[wkey] = [np.corrcoef(stats[skey], bms[wkey])[0][1] 
                     for skey in stats.columns]
    df = pd.DataFrame(dic, index=stats.columns)
    df.apply(lambda s: s.apply(lambda v: int(v*100) if abs(v)>=.80 else ""))
    return df

#calc_percent(bms1)
df = get_stats_correlations(
    calc_stats_percent(stats),
    calc_bmwahl_percent(bms1),
)
df.apply(lambda s: s.apply(lambda v: int(v*100) if abs(v)>=.50 else ""))
#stats["Arbeitslose insgesamt(2016)"] /= stats["Einwohner (Hauptwohnung) insgesamt(2016)"]
#calc_stats_percent(stats)
#calc_bmwahl_percent(bms1)
#print(bms1["n"])
#print(stats["Einwohner (Hauptwohnung) insgesamt(2016)"])
#np.corrcoef(stats["Einwohner (Hauptwohnung) insgesamt(2016)"], bms1["n"])
        

/home/bergi/prog/python/wahl/env/lib/python3.5/site-packages/numpy/lib/function_base.py:3183: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/bergi/prog/python/wahl/env/lib/python3.5/site-packages/numpy/lib/function_base.py:3184: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


,n,nw,nu,ng,CDU,LINKE,SPD,AFD,GRÜNE,FDP,πRATEN,SANDRO,ARNE
Arbeitslose insgesamt(2016),,-50,,,,,,,,,,,
Arbeitslose - Männer(2016),,,,,,,,,,,,,
Arbeitslose - Frauen(2016),61,-55,,,,57,,,,,,,
Arbeitslose - Deutsche(2016),,,,,,,,,,,,,
Arbeitslose - Ausländer(2016),50,,,,,,,,,,,,
Arbeitslose - unter 20 Jahre alt(2016),62,,,,,58,,,,,,,
Arbeitslose - 20 bis unter 25 Jahre alt(2016),,,,,,,,,,,,,
Arbeitslose - 55 Jahre und älter(2016),,,,,,,,,,,,,
SGB II - Arbeitslose insgesamt(2016),,,,,,,,,,,,,
SGB II - Arbeitslose - Männer(2016),,,,,,,,,,,,,


In [11]:
(stats.add(bms1,fill_value=0)).corr().apply(lambda s: s.apply(lambda v: int(v*100) if abs(v)>=.80 else ""))

,AFD,ARNE,Ackerland(2016),Alleinerziehende Frauen mit 1 Kind(2016),Alleinerziehende Frauen mit 2 Kindern(2016),Alleinerziehende Frauen mit 3 oder mehr Kindern(2016),Alleinerziehende Männer mit 1 Kind(2016),Alleinerziehende Männer mit 2 Kindern(2016),Alleinerziehende Männer mit 3 oder mehr Kindern(2016),Arbeitslose - 20 bis unter 25 Jahre alt(2016),...,Wohnungen mit 4 Räumen(2016),Wohnungen mit 5 Räumen(2016),Wohnungen mit 6 Räumen(2016),Wohnungen mit 7 und mehr Räumen(2016),n,ng,nu,nw,sonstige Fläche(2016),πRATEN
AFD,100,87,,94,93,82,95,82,,86,...,94,93,,,93,87,89,87,,
ARNE,87,100,,89,85,,88,87,,,...,87,88,,,92,93,86,93,,85
Ackerland(2016),,,100,,,,,,,,...,,,,,,,,,,
Alleinerziehende Frauen mit 1 Kind(2016),94,89,,100,96,82,96,82,,91,...,98,94,,,94,88,86,88,,
Alleinerziehende Frauen mit 2 Kindern(2016),93,85,,96,100,86,92,80,,90,...,97,94,,,93,87,90,87,,
Alleinerziehende Frauen mit 3 oder mehr Kindern(2016),82,,,82,86,100,82,,,90,...,85,82,,,,,,,,
Alleinerziehende Männer mit 1 Kind(2016),95,88,,96,92,82,100,,,88,...,95,94,,,95,89,89,89,,
Alleinerziehende Männer mit 2 Kindern(2016),82,87,,82,80,,,100,,,...,81,81,,,88,91,,91,,83
Alleinerziehende Männer mit 3 oder mehr Kindern(2016),,,,,,,,,100,,...,,,,,,,,,,
Arbeitslose - 20 bis unter 25 Jahre alt(2016),86,,,91,90,90,88,,,100,...,92,86,,,83,,81,,,


In [12]:
s = bm1["n"]
s.clip(1000)

Bezirk
001 Dezernat Stadtentwicklung                                 1685
002 WE Catering / Raum OG                                     1144
003 Nordschule / Raum 00_03                                   1000
004 WE Catering / Raum EG                                     1682
005 Volksbad-Anbau                                            1543
006 Seniorenwohnen Am Villengang                              1000
007 Technologie- und Innovationspark Jena                     1270
008 Feuerwehrgerätehaus Lichtenhain                           1000
009 Jenaplan-Schule / Raum 01.00.15                           1000
010 Jenaplan-Schule / Raum 01.00.14                           1532
011 Jenaplan-Schule / Raum 01.00.13                           1374
012 Aktion Wandlungswelten / EG Zimmer 2                      1000
013 Aktion Wandlungswelten / EG Zimmer 3                      1022
014 IGS Grete Unrein / Raum 00-14                             1397
015 IGS Grete Unrein / Raum 00-15                      